In [0]:
from pyspark.sql.functions import col, current_timestamp, lit, to_date

In [0]:
partition_name = 'partition_date'
partition_date = '20250828'
input_file_name = f"sc_raw_customer_journey_{partition_date}.csv"
path = f"/Volumes/ctl_central_published/sc_volumes/ext_vol_upload/dgai/{input_file_name}"

In [0]:
_ctl = 'ctl_central_published'
_schema = 'sc_bz_source_name'
_table = 'dt_customer_journey_daily'

In [0]:
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "false")
      .option("multiLine", "true")
      .option("quote", '"')
      .option("escape", '"')
      .csv(path))

df = (df.select([col(c).cast("string").alias(c) for c in df.columns])
        .withColumn("_source_file", lit(path))
        .withColumn("_ingest_ts", current_timestamp())
        .withColumn(partition_name, to_date(lit(partition_date), "yyyyMMdd"))
)

In [0]:
df.display()

In [0]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

(df.write
   .format("delta")
   .mode("overwrite")
   .partitionBy(partition_name)
   .saveAsTable(f"{_ctl}.{_schema}.{_table}"))